# 파이썬을 이용한 비트코인 자동매매


In [2]:
!pip install pybithumb

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.1/247.1 kB 10.3 MB/s eta 0:00:00
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1257 sha256=716fe65a399d0512583330e65fe0f8d8dffb89096e0cd2f106a1e1735e049108
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


## CH07 백테스팅
- **백테스팅: 과거 데이터를 사용해서 투자 전략이 어느 정도의 수익률이 나는지 확인하는 과정**

### 7-1. 백테스팅을 위한 데이터 준비하기

#### 7-1-1. 가상화폐 일봉 데이터 얻기
- pybitumb 모듈의 get_ohlcv() 함수 사용

In [3]:
import pybithumb

df = pybithumb.get_ohlcv("BTC")
print(df.tail())

                           open        high         low       close  \
time                                                                  
2023-11-15 00:00:00  49072000.0  49089000.0  46795000.0  48238000.0   
2023-11-16 00:00:00  48237000.0  50646000.0  48229000.0  49212000.0   
2023-11-17 00:00:00  49212000.0  49491000.0  48102000.0  48683000.0   
2023-11-18 00:00:00  48683000.0  49430000.0  48592000.0  49128000.0   
2023-11-19 21:00:00  49128000.0  49432000.0  48957000.0  49082000.0   

                          volume  
time                              
2023-11-15 00:00:00  5555.061989  
2023-11-16 00:00:00  5866.758722  
2023-11-17 00:00:00  5665.446334  
2023-11-18 00:00:00  3831.054192  
2023-11-19 21:00:00  2584.815074  


#### 7-1-2. DataFrame 객체를 엑셀로 저장하기

In [12]:
import pybithumb

df = pybithumb.get_ohlcv("BTC")
df.to_excel("btc.xlsx")

df.head()

,open,high,low,close,volume
time,,,,,
2013-12-27,737000.0,755000.0,737000.0,755000.0,3.780000
2013-12-28,750000.0,750000.0,750000.0,750000.0,12.000000
2013-12-29,750000.0,750000.0,728000.0,739000.0,19.058000
2013-12-30,740000.0,772000.0,740000.0,768000.0,9.488973
2013-12-31,768000.0,800000.0,763000.0,768000.0,18.650350


### 7-2. 변동성 돌파 전략 백테스팅

#### 7-2-1. 레인지 계산하기
- 레인지 = 전일 고가 - 전일 저가
    - 래리 윌리엄스의 변동성 돌파 전략에서 사용

- 자동 매매 프로그램에서는 거래일 하루 전날에 대해서 레인지를 계산하지만, 백테스팅은 모든 거래일을 기준으로 레인지를 계산해야 함.

In [11]:
import pybithumb

df = pybithumb.get_ohlcv("BTC")
df['range'] = (df['high'] - df['low']) * 0.5
df.to_excel("btc.xlsx")

df.head()

,open,high,low,close,volume,range
time,,,,,,
2013-12-27,737000.0,755000.0,737000.0,755000.0,3.780000,9000.0
2013-12-28,750000.0,750000.0,750000.0,750000.0,12.000000,0.0
2013-12-29,750000.0,750000.0,728000.0,739000.0,19.058000,11000.0
2013-12-30,740000.0,772000.0,740000.0,768000.0,9.488973,16000.0
2013-12-31,768000.0,800000.0,763000.0,768000.0,18.650350,18500.0


#### 7-2-2. 목표가 계산하기
- 목표가: 거래일 당일 시가(open) + 전일 레인지
- `df['open'] + df['range'].shift(1)`로 `range` 컬럼의 값을 1행 내려준 후, `df['open']` 컬럼과 더한 값을 `target` 컬럼에 저장함.

In [9]:
import pybithumb

df = pybithumb.get_ohlcv("BTC")
df['range'] = (df['high'] - df['low']) * 0.5
df['target'] = df['open'] + df['range'].shift(1)
df.to_excel("btc.xlsx")

df.head()

,open,high,low,close,volume,range,target
time,,,,,,,
2013-12-27,737000.0,755000.0,737000.0,755000.0,3.780000,9000.0,NaN
2013-12-28,750000.0,750000.0,750000.0,750000.0,12.000000,0.0,759000.0
2013-12-29,750000.0,750000.0,728000.0,739000.0,19.058000,11000.0,750000.0
2013-12-30,740000.0,772000.0,740000.0,768000.0,9.488973,16000.0,751000.0
2013-12-31,768000.0,800000.0,763000.0,768000.0,18.650350,18500.0,784000.0


In [10]:
import pybithumb

df = pybithumb.get_ohlcv("BTC")
df['range'] = (df['high'] - df['low']) * 0.5
df['range_shift1'] = df['range'].shift(1) # df['range'].shift(1) 결과를 포함
df['target'] = df['open'] + df['range'].shift(1)
df.to_excel("btc.xlsx")

df.head()

,open,high,low,close,volume,range,range_shift1,target
time,,,,,,,,
2013-12-27,737000.0,755000.0,737000.0,755000.0,3.780000,9000.0,NaN,NaN
2013-12-28,750000.0,750000.0,750000.0,750000.0,12.000000,0.0,9000.0,759000.0
2013-12-29,750000.0,750000.0,728000.0,739000.0,19.058000,11000.0,0.0,750000.0
2013-12-30,740000.0,772000.0,740000.0,768000.0,9.488973,16000.0,11000.0,751000.0
2013-12-31,768000.0,800000.0,763000.0,768000.0,18.650350,18500.0,16000.0,784000.0


#### 7-2-3. 매수, 매도 그리고 수익률
- 각 거래일별로 매매 조건을 확인하고 그때의 수익률을 계산함.
- 변동성 돌파전략에서 매수는 각 거래의 고가(high)가 목표가(target) 이상이면 수행된 것으로 간주함.

```
numpy.where(조건, 조건이 참일 때의 값, 조건이 거짓일 때의 값)
```

In [13]:
import numpy as np
from pandas import DataFrame

data = {'빗썸': [100, 100, 100],
        '코빗': [90, 110, 120]}

df = DataFrame(data)
df['최저가'] = np.where(df['빗썸'] < df['코빗'], '빗썸', '코빗')
df.to_excel("거래소.xlsx")

df.head()

,빗썸,코빗,최저가
0,100,90,코빗
1,100,110,빗썸
2,100,120,빗썸


- 고가가 목표가보다 크면 매수 조건에 해당하고, 그 때의 수익률은 '매도가/매수가'임.

In [14]:
import pybithumb
import numpy as np

df = pybithumb.get_ohlcv("BTC")
df['range'] = (df['high'] - df['low']) * 0.5
df['target'] = df['open'] + df['range'].shift(1)

df['ror'] = np.where(df['high'] > df['target'],
                     df['close'] / df['target'],
                     1)

df.to_excel("trade.xlsx")

df.head()

,open,high,low,close,volume,range,target,ror
time,,,,,,,,
2013-12-27,737000.0,755000.0,737000.0,755000.0,3.780000,9000.0,NaN,1.000000
2013-12-28,750000.0,750000.0,750000.0,750000.0,12.000000,0.0,759000.0,1.000000
2013-12-29,750000.0,750000.0,728000.0,739000.0,19.058000,11000.0,750000.0,1.000000
2013-12-30,740000.0,772000.0,740000.0,768000.0,9.488973,16000.0,751000.0,1.022636
2013-12-31,768000.0,800000.0,763000.0,768000.0,18.650350,18500.0,784000.0,0.979592



- 백테스팅에서 수익률을 퍼센트(%)로 계산하는 것보다 **몇 배(ratio)**로 표현하는 것이 더 좋음.
- **기간수익률(HPR: Holding Period Return): 각 거래일에 대해 수익률을 계산한 후, 모든 거래의 수익률을 곱함.**

In [15]:
import pybithumb
import numpy as np

df = pybithumb.get_ohlcv("BTC")
df['range'] = (df['high'] - df['low']) * 0.5
df['target'] = df['open'] + df['range'].shift(1)

df['ror'] = np.where(df['high'] > df['target'],
                     df['close'] / df['target'],
                     1)

ror = df['ror'].cumprod()[-2] # cumprod()는 Series 객체의 모든 값을 곱해줌.
print(ror) # 약 122배의 누적 수익률을 얻을 수 있음,

122.7383422412149


- 2018년도만 투자했을 때의 누적 수익률

In [16]:
import pybithumb
import numpy as np

df = pybithumb.get_ohlcv("BTC")
df = df['2018']

df['range'] = (df['high'] - df['low']) * 0.5
df['target'] = df['open'] + df['range'].shift(1)

df['ror'] = np.where(df['high'] > df['target'],
                     df['close'] / df['target'],
                     1)

ror = df['ror'].cumprod()[-2]
print(ror) # 1.18배 수익률

1.1817568351455974


<ipython-input-16-a7916658fb57>:5: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  df = df['2018']


#### 7-2-4. 수수료 및 슬리피지
- **수수료: 빗썸 거래소에서 가상화폐 매매시, 0.15%의 수수료가 적용됨.**
- **슬리피지: 시장가 주문시,  거래량 부족으로 인해 생각하는 가격대보다 더 비싸게 매수되거나 더 싸게 매도될 때 발생하는 비용**

|항목|금액|
|------|---|
|투자원금|10,000,000|
|매수체결가|6,900,000|
|매도체결가|7,200,000|
|매수체결수량|1.449275362|
|매수체결수량(수수료 제외)|1.447101449|
|매도체결수량|1.447101449|
|거래금액|10,419,130.43|
|거래금액(수수료 제외)|10,403,501.74|
|체결가 기준 수익률(배)|1.0434782|
|실제 수익률(배)|1.0403501|
|수익률 차이|0.003128|

In [17]:
import pybithumb
import numpy as np

df = pybithumb.get_ohlcv("BTC")
df['range'] = (df['high'] - df['low']) * 0.5
df['target'] = df['open'] + df['range'].shift(1)

fee = 0.0032 # 수수료
df['ror'] = np.where(df['high'] > df['target'],
                     df['close'] / df['target'] - fee,
                     1)

ror = df['ror'].cumprod()[-2]
print(ror)

1.263304823492166


#### 7-2-5. 가장 좋은 k 값 구하기
- k 값: 레인지르르 구할 때 전날의 고가에서 저가를 뺀 후 곱해주는 값

In [18]:
import pybithumb
import numpy as np


def get_ror(k=0.5):
    df = pybithumb.get_ohlcv("BTC")
    df['range'] = (df['high'] - df['low']) * k
    df['target'] = df['open'] + df['range'].shift(1)

    fee = 0.0032
    df['ror'] = np.where(df['high'] > df['target'],
                         df['close'] / df['target'] - fee,
                         1)

    ror = df['ror'].cumprod()[-2]
    return ror


for k in np.arange(0.1, 1.0, 0.1):
    ror = get_ror(k)
    print("%.1f %f" % (k, ror)) # 0.8일때 가장 높은 수익률

0.1 0.000780
0.2 0.005199
0.3 0.034387
0.4 0.372427
0.5 1.263305
0.6 3.643424
0.7 6.445955
0.8 8.346937
0.9 5.847689


#### 7-2-6. MDD(Maximum Draw Down) 계산하기
- MDD(Maximum Draw Down): 투자 기간 중에 포트폴리오의 전 고점에서 저점까지의 최대 누적 손실
 - MDD = (max - low) / max * 100
```
EX) 비트코인에 100만원 투자, 투자 기간동안 자산 200만원 -> 50만원 -> 300만원.
MDD = (200-50)/200 * 100
```

- MDD를 구하기 위해 거래일별로 기간수익률 필요

In [20]:
import pybithumb
import numpy as np

df = pybithumb.get_ohlcv("BTC")
df['range'] = (df['high'] - df['low']) * 0.5
df['target'] = df['open'] + df['range'].shift(1)

fee = 0.0032
df['ror'] = np.where(df['high'] > df['target'],
                     df['close'] / df['target'] - fee,
                     1)

df['hpr'] = df['ror'].cumprod()
df.to_excel("btc.xlsx")

df.head()

,open,high,low,close,volume,range,target,ror,hpr
time,,,,,,,,,
2013-12-27,737000.0,755000.0,737000.0,755000.0,3.780000,9000.0,NaN,1.000000,1.000000
2013-12-28,750000.0,750000.0,750000.0,750000.0,12.000000,0.0,759000.0,1.000000,1.000000
2013-12-29,750000.0,750000.0,728000.0,739000.0,19.058000,11000.0,750000.0,1.000000,1.000000
2013-12-30,740000.0,772000.0,740000.0,768000.0,9.488973,16000.0,751000.0,1.019436,1.019436
2013-12-31,768000.0,800000.0,763000.0,768000.0,18.650350,18500.0,784000.0,0.976392,0.995369


- 각 거래일에 대한 기간 수익률을 사용해서 거래일마다 낙폭을 구하고 낙폭 중 최대값을 찾아 MDD를 구함.

In [21]:
import pybithumb
import numpy as np

df = pybithumb.get_ohlcv("BTC")
df['range'] = (df['high'] - df['low']) * 0.5
df['target'] = df['open'] + df['range'].shift(1)

fee = 0.0032
df['ror'] = np.where(df['high'] > df['target'],
                     df['close'] / df['target'] - fee,
                     1)

df['hpr'] = df['ror'].cumprod()
df['dd'] = (df['hpr'].cummax() - df['hpr']) / df['hpr'].cummax() * 100 # cummax()로 전고점 값을 얻어옴
print("MDD(%): ", df['dd'].max())
df.to_excel("dd.xlsx")

df.head()

MDD(%):  75.8238846772885


,open,high,low,close,volume,range,target,ror,hpr,dd
time,,,,,,,,,,
2013-12-27,737000.0,755000.0,737000.0,755000.0,3.780000,9000.0,NaN,1.000000,1.000000,0.000000
2013-12-28,750000.0,750000.0,750000.0,750000.0,12.000000,0.0,759000.0,1.000000,1.000000,0.000000
2013-12-29,750000.0,750000.0,728000.0,739000.0,19.058000,11000.0,750000.0,1.000000,1.000000,0.000000
2013-12-30,740000.0,772000.0,740000.0,768000.0,9.488973,16000.0,751000.0,1.019436,1.019436,0.000000
2013-12-31,768000.0,800000.0,763000.0,768000.0,18.650350,18500.0,784000.0,0.976392,0.995369,2.360816


### 7-3. 변동성 돌파+상승장 전략 백테스팅

#### 7-3-1. 변동성 돌파+상승장 전략 백테스팅
- 5일 이동평균을 사용해 상승장/하락장을 판단한 후, 상승장이면서 변동성 돌파 전략의 조건을 만족할 때 매수함.

In [22]:
import pybithumb
import numpy as np

df = pybithumb.get_ohlcv("BTC")

df['ma5'] = df['close'].rolling(window=5).mean().shift(1)
df['range'] = (df['high'] - df['low']) * 0.5
df['target'] = df['open'] + df['range'].shift(1)
df['bull'] = df['open'] > df['ma5'] # 상승장 여부

fee = 0.0032
df['ror'] = np.where((df['high'] > df['target']) & df['bull'],
                      df['close'] / df['target'] - fee,
                      1)

df['hpr'] = df['ror'].cumprod()
df['dd'] = (df['hpr'].cummax() - df['hpr']) / df['hpr'].cummax() * 100
print("MDD: ", df['dd'].max())
print("HPR: ", df['hpr'][-2])
df.to_excel("larry_ma.xlsx")

df.head()

MDD:  41.07120419932468
HPR:  18.065962833100635


,open,high,low,close,volume,ma5,range,target,bull,ror,hpr,dd
time,,,,,,,,,,,,
2013-12-27,737000.0,755000.0,737000.0,755000.0,3.780000,NaN,9000.0,NaN,False,1.0,1.0,0.0
2013-12-28,750000.0,750000.0,750000.0,750000.0,12.000000,NaN,0.0,759000.0,False,1.0,1.0,0.0
2013-12-29,750000.0,750000.0,728000.0,739000.0,19.058000,NaN,11000.0,750000.0,False,1.0,1.0,0.0
2013-12-30,740000.0,772000.0,740000.0,768000.0,9.488973,NaN,16000.0,751000.0,False,1.0,1.0,0.0
2013-12-31,768000.0,800000.0,763000.0,768000.0,18.650350,NaN,18500.0,784000.0,False,1.0,1.0,0.0


#### 7-3-2. 2018년도 기간수익률이 높은 코인찾기
- 빗썸에서 거래되는 모든 코인에 대해 '변동성 돌파+상승장 전략'의 기간수익률이 높은 코인 5개 찾기

In [25]:
import pybithumb
import numpy as np


def get_hpr(ticker):
    try:
        df = pybithumb.get_ohlcv(ticker)
        df = df.loc['2018']

        df['ma5'] = df['close'].rolling(window=5).mean().shift(1)
        df['range'] = (df['high'] - df['low']) * 0.5
        df['target'] = df['open'] + df['range'].shift(1)
        df['bull'] = df['open'] > df['ma5']

        fee = 0.0032
        df['ror'] = np.where((df['high'] > df['target']) & df['bull'],
                              df['close'] / df['target'] - fee,
                              1)

        df['hpr'] = df['ror'].cumprod()
        df['dd'] = (df['hpr'].cummax() - df['hpr']) / df['hpr'].cummax() * 100
        return df['hpr'][-2]

    # 2018년 데이터가 없는 코인이 존재해서 예외처리
    except KeyError:
        return None



tickers = pybithumb.get_tickers()

hprs = []

for ticker in tickers:
    hpr = get_hpr(ticker)
    if hpr is not None:
        hprs.append((ticker, hpr))

sorted_hprs = sorted(hprs, key=lambda x:x[1])
print(sorted_hprs[-5:])

[('WAXP', 2.1640363536847405), ('XRP', 2.469533973902334), ('LRC', 2.9260451669193657), ('SNT', 4.0216481243550755), ('POWR', 5.65256529365624)]
